In [7]:
import pandas as pd
from numpy.random import rand
from yaml import load, Loader

from planktonsdm.predict import predict
from planktonsdm.functions import example_data
from sklearn.preprocessing import OneHotEncoder

In [8]:
#If you want to debug the package without repackaging, you can read files locally:

import sys
sys.path.insert(0, '/home/phyto/planktonSDM/planktonsdm/')
from predict import predict
from functions import example_data

In [9]:
with open('/home/phyto/planktonSDM/configuration/2-phase_ensemble.yml', 'r') as f:
    model_config = load(f, Loader=Loader)

d = pd.read_csv("/home/phyto/planktonSDM/data/abundances_environment.csv")
species = 'Umbellosphaera tenuis'

d.dropna(subset=species, inplace=True)
d.dropna(subset='FID', inplace=True)

features = model_config['predictors']

X_train = d[features]
y = d[species]

X_predict = pd.read_csv("/home/phyto/planktonSDM/data/envdata_april_surface.csv")
X_predict.set_index(["time", "depth", "lat", "lon"], inplace=True)

#drop rows with FIDs not found in original dataset:
X_predict = X_predict[X_predict['FID'].isin(X_train['FID'])]
X_predict.set_index([], inplace=True)
enc = OneHotEncoder()
regions  = enc.fit_transform(X_train[['FID']])
X_train[enc.categories_[0]] = regions.toarray()
X_train.columns = X_train.columns.astype(str)
X_train.drop(columns=['FID'], inplace=True)

regions  = enc.fit_transform(X_predict[['FID']])
X_predict[enc.categories_[0]] = regions.toarray()
X_predict.columns = X_predict.columns.astype(str)
X_predict.drop(columns=['FID'], inplace=True)


/tmp/ipykernel_22035/1876859278.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[enc.categories_[0]] = regions.toarray()
/tmp/ipykernel_22035/1876859278.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[enc.categories_[0]] = regions.toarray()
/tmp/ipykernel_22035/1876859278.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [10]:
with open('/home/phyto/planktonSDM/configuration/1-phase_rf.yml', 'r') as f:
    model_config = load(f, Loader=Loader)

m = predict(X_train, y, X_predict, model_config)
m.make_prediction()

number of models in ensemble:1
predicting classifier
finished
execution time: 0.6199193000793457 seconds


In [11]:
with open('/home/phyto/planktonSDM/configuration/2-phase_ensemble.yml', 'r') as f:
    model_config = load(f, Loader=Loader)
m = predict(X_train, y, X_predict, model_config)
m.make_prediction()

upsampling = True
number of models in ensemble:3
predicting zero-inflated regressor


/home/phyto/anaconda3/envs/my-geopandas-env-2/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/phyto/anaconda3/envs/my-geopandas-env-2/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


exporting rf prediction to: /home/phyto/CoccoML/ModelOutput/rf/predictions/
predicting zero-inflated regressor


/home/phyto/anaconda3/envs/my-geopandas-env-2/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/phyto/anaconda3/envs/my-geopandas-env-2/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


exporting xgb prediction to: /home/phyto/CoccoML/ModelOutput/xgb/predictions/
predicting zero-inflated regressor


/home/phyto/anaconda3/envs/my-geopandas-env-2/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/phyto/anaconda3/envs/my-geopandas-env-2/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


exporting knn prediction to: /home/phyto/CoccoML/ModelOutput/knn/predictions/
[0.35082571417379915, 0.3236194136190801, 0.32555487220712076]
[('rf', ZeroInflatedRegressor(classifier=Pipeline(steps=[('preprocessor',
                                                  ColumnTransformer(transformers=[('num',
                                                                                   Pipeline(steps=[('scaler',
                                                                                                    StandardScaler())]),
                                                                                   array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43]))])),
                                                 ('estimator',
                                                  RandomForestClassifier(max_depth=50,
                            